# Self-Organizing Maps (SOMs) Notebook
## Data extraction for composites

**Notebook by Maria J. Molina (NCAR) and Alice DuVivier (NCAR).**

This Notebook reads in data from the CESM2-LE for a user-specified variable. It subsets the data to be just around Antarctica to create composites from.

In [1]:
# Needed imports
import xarray as xr
import cftime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from datetime import timedelta
from itertools import product
import geocat.comp as gcomp

import dask
import intake
from distributed import Client
from ncar_jobqueue import NCARCluster

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# spin up dask cluster

import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=36, # The number of cores you want
    memory='300 GB', # Amount of memory
    processes=9, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=36:mem=300GB', # Specify resources
    project='P93300665', # Input your project ID here
    walltime='06:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)
# Scale up
cluster.scale(jobs=8)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

cluster = NCARCluster(memory="100GB", walltime='8:00:00', cores=4, processes=2, resource_spec='select=1:ncpus=2:mem=100GB')
Each worker has 100GB, resource_spec is assigning this. 

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.57:39654,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Section 1: Load and get correct training data

In [4]:
# set some info for the CESM2-LE data
# set: variable to test, the location of the data, which ensemble member
# list the variables to load
var_in_1 = 'T'
var_in_2 = 'TS'
var_in_3 = 'PS'

 # do not want smbb data
forcing = 'cmip6'
# only daily data
freq = 'month_1'

### Load in the data

In [5]:
catalog_file = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'

cat = intake.open_esm_datastore(catalog_file)

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/intake_esm/core.py:84: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  self._df, self.catalog_file = _fetch_catalog(self.esmcol_data, esmcol_obj, csv_kwargs)


In [6]:
cat

,unique
component,6
stream,26
case,200
member_id,100
variable,1906
start_time,157
end_time,180
time_range,163
long_name,1800
units,184


In [7]:
subset_1 = cat.search(variable=var_in_1, forcing_variant=forcing, frequency=freq)
subset_2 = cat.search(variable=var_in_2, forcing_variant=forcing, frequency=freq)
subset_3 = cat.search(variable=var_in_3, forcing_variant=forcing, frequency=freq)

In [8]:
#subset
subset_1.df.head()

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,atm,cam.h0,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,1850-01,1859-12,185001-185912,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
1,atm,cam.h0,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,1860-01,1869-12,186001-186912,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
2,atm,cam.h0,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,1870-01,1879-12,187001-187912,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
3,atm,cam.h0,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,1880-01,1889-12,188001-188912,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
4,atm,cam.h0,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,1890-01,1899-12,189001-189912,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1


In [9]:
subset_1

,unique
component,1
stream,1
case,100
member_id,50
variable,1
start_time,26
end_time,26
time_range,26
long_name,1
units,1


In [10]:
# make arrays of half (25) of the CESM2-LE members 
# select every other from the large ensemble of both macro and micro starts
# note that the naming of the files (YYYY.#### e.g. 1001.001) doesn't match the member_id directly, 
# but the ensemble number (### e.g. 001) does match the member_id field r? directly. So use this to search

# set list of members from the dataset
member_ids = subset_1.df.member_id.unique()

# set list of members to KEEP
keep_list = ['r1i', 'r3i', 'r5i','r7i', 'r9i']


In [11]:
member_keep = [] # make a list to fill

for member in keep_list:
    for member_id in member_ids:
        if member in member_id:
            member_keep.append(member_id)

In [12]:
#check that we're keeping the right ones
member_keep

['r1i1001p1f1',
 'r1i1231p1f1',
 'r1i1251p1f1',
 'r1i1281p1f1',
 'r1i1301p1f1',
 'r3i1041p1f1',
 'r3i1231p1f1',
 'r3i1251p1f1',
 'r3i1281p1f1',
 'r3i1301p1f1',
 'r5i1081p1f1',
 'r5i1231p1f1',
 'r5i1251p1f1',
 'r5i1281p1f1',
 'r5i1301p1f1',
 'r7i1121p1f1',
 'r7i1231p1f1',
 'r7i1251p1f1',
 'r7i1281p1f1',
 'r7i1301p1f1',
 'r9i1161p1f1',
 'r9i1231p1f1',
 'r9i1251p1f1',
 'r9i1281p1f1',
 'r9i1301p1f1']

In [13]:
# now reduce subset based on just the members to keep
subset_1 = subset_1.search(member_id=member_keep)
subset_2 = subset_2.search(member_id=member_keep)
subset_3 = subset_3.search(member_id=member_keep)

In [14]:
%%time
#actually load the data we selected into a dataset
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_1 = subset_1.to_dataset_dict(cdf_kwargs={'chunks': {'time':50}, 'decode_times': True})

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_2 = subset_2.to_dataset_dict(cdf_kwargs={'chunks': {'time':50}, 'decode_times': True})
    
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_3 = subset_3.to_dataset_dict(cdf_kwargs={'chunks': {'time':50}, 'decode_times': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


CPU times: user 29.2 s, sys: 803 ms, total: 30 s
Wall time: 47 s


In [15]:
# print names of the dataset keys, which refer to each of the ensembles loaded
dsets_1.keys()

dict_keys(['atm.ssp370.cam.h0.cmip6.T', 'atm.historical.cam.h0.cmip6.T'])

In [16]:
# load in the historical and future datasets

historicals_1 = []
futures_1 = []
for key in sorted(dsets_1.keys()):
    if 'historical' in key:
        historicals_1.append(dsets_1[key])
        print(key)
    elif 'ssp370' in key:
        futures_1.append(dsets_1[key])
        print(key)

historicals_2 = []
futures_2 = []
for key in sorted(dsets_2.keys()):
    if 'historical' in key:
        historicals_2.append(dsets_2[key])
        print(key)
    elif 'ssp370' in key:
        futures_2.append(dsets_2[key])
        print(key)

historicals_3 = []
futures_3 = []
for key in sorted(dsets_3.keys()):
    if 'historical' in key:
        historicals_3.append(dsets_3[key])
        print(key)
    elif 'ssp370' in key:
        futures_3.append(dsets_3[key])
        print(key)

atm.historical.cam.h0.cmip6.T
atm.ssp370.cam.h0.cmip6.T
atm.historical.cam.h0.cmip6.TS
atm.ssp370.cam.h0.cmip6.TS
atm.historical.cam.h0.cmip6.PS
atm.ssp370.cam.h0.cmip6.PS


In [17]:
# Now put these into an array by member_id
historical_ds = xr.concat(historicals_1, dim='member_id')
future_ds = xr.concat(futures_1, dim='member_id')

# note that the historical and future xarray datasets have the same coordinates and dimensions *except* time, 
# so we need to concatenate over time
ds_1 = xr.concat([historical_ds,future_ds],dim='time')

In [18]:
# Now put these into an array by member_id
historical_ds = xr.concat(historicals_2, dim='member_id')
future_ds = xr.concat(futures_2, dim='member_id')

# note that the historical and future xarray datasets have the same coordinates and dimensions *except* time, 
# so we need to concatenate over time
ds_2 = xr.concat([historical_ds,future_ds],dim='time')

In [19]:
# Now put these into an array by member_id
historical_ds = xr.concat(historicals_3, dim='member_id')
future_ds = xr.concat(futures_3, dim='member_id')

# note that the historical and future xarray datasets have the same coordinates and dimensions *except* time, 
# so we need to concatenate over time
ds_3 = xr.concat([historical_ds,future_ds],dim='time')

## Section 2: Subset the times we need

In [20]:
ds_1.time

<xarray.DataArray 'time' (time: 3012)>
array([cftime.DatetimeNoLeap(1850, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 4, 1, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2100, 11, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2100, 12, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2101, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-02-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    long_name:  time
    bounds:     time_bnds

In [24]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
ds_1['time'] = ds_1.time_bnds.load().mean(dim='nbnd').isel(member_id=0)
ds_2['time'] = ds_2.time_bnds.load().mean(dim='nbnd').isel(member_id=0)
ds_3['time'] = ds_3.time_bnds.load().mean(dim='nbnd').isel(member_id=0)

In [25]:
ds_1.time

<xarray.DataArray 'time' (time: 3012)>
array([cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(2100, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2100, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2100, 12, 16, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    long_name:  time interval endpoints

In [31]:
# keep just years greater than 1980 and less than 2080 
yy_st = "1980"
yy_ed = "2080"
ds_1 = ds_1.sel(time=slice(yy_st, yy_ed))
ds_2 = ds_2.sel(time=slice(yy_st, yy_ed))
ds_3 = ds_3.sel(time=slice(yy_st, yy_ed))

In [32]:
ds_1.time.dt.month

<xarray.DataArray 'month' (time: 1212)>
array([ 1,  2,  3, ..., 10, 11, 12])
Coordinates:
  * time     (time) object 1980-01-16 12:00:00 ... 2080-12-16 12:00:00

In [33]:
# keep just times corresponding to winter (SH: all times between april and sept)
ds_1 = ds_1.isel(time=ds_1.time.dt.month.isin([7,8,9]))
ds_2 = ds_2.isel(time=ds_2.time.dt.month.isin([7,8,9]))
ds_3 = ds_3.isel(time=ds_3.time.dt.month.isin([7,8,9]))

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [34]:
ds_1.time

<xarray.DataArray 'time' (time: 303)>
array([cftime.DatetimeNoLeap(1980, 7, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1980, 8, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1980, 9, 16, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2080, 7, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2080, 8, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2080, 9, 16, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1980-07-16 12:00:00 ... 2080-09-16 00:00:00
Attributes:
    long_name:  time interval endpoints

## Section 3: Drop the lats that we don't need

In [35]:
# look at latitudes
ds_1.lat

<xarray.DataArray 'lat' (lat: 192)>
array([-90.      , -89.057592, -88.115183, -87.172775, -86.230366, -85.287958,
       -84.34555 , -83.403141, -82.460733, -81.518325, -80.575916, -79.633508,
       -78.691099, -77.748691, -76.806283, -75.863874, -74.921466, -73.979058,
       -73.036649, -72.094241, -71.151832, -70.209424, -69.267016, -68.324607,
       -67.382199, -66.439791, -65.497382, -64.554974, -63.612565, -62.670157,
       -61.727749, -60.78534 , -59.842932, -58.900524, -57.958115, -57.015707,
       -56.073298, -55.13089 , -54.188482, -53.246073, -52.303665, -51.361257,
       -50.418848, -49.47644 , -48.534031, -47.591623, -46.649215, -45.706806,
       -44.764398, -43.82199 , -42.879581, -41.937173, -40.994764, -40.052356,
       -39.109948, -38.167539, -37.225131, -36.282723, -35.340314, -34.397906,
       -33.455497, -32.513089, -31.570681, -30.628272, -29.685864, -28.743455,
       -27.801047, -26.858639, -25.91623 , -24.973822, -24.031414, -23.089005,
       -22.146597, -21.204188, -20.26178 , -19.319372, -18.376963, -17.434555,
       -16.492147, -15.549738, -14.60733 , -13.664921, -12.722513, -11.780105,
       -10.837696,  -9.895288,  -8.95288 ,  -8.010471,  -7.068063,  -6.125654,
        -5.183246,  -4.240838,  -3.298429,  -2.356021,  -1.413613,  -0.471204,
         0.471204,   1.413613,   2.356021,   3.298429,   4.240838,   5.183246,
         6.125654,   7.068063,   8.010471,   8.95288 ,   9.895288,  10.837696,
        11.780105,  12.722513,  13.664921,  14.60733 ,  15.549738,  16.492147,
        17.434555,  18.376963,  19.319372,  20.26178 ,  21.204188,  22.146597,
        23.089005,  24.031414,  24.973822,  25.91623 ,  26.858639,  27.801047,
        28.743455,  29.685864,  30.628272,  31.570681,  32.513089,  33.455497,
        34.397906,  35.340314,  36.282723,  37.225131,  38.167539,  39.109948,
        40.052356,  40.994764,  41.937173,  42.879581,  43.82199 ,  44.764398,
        45.706806,  46.649215,  47.591623,  48.534031,  49.47644 ,  50.418848,
        51.361257,  52.303665,  53.246073,  54.188482,  55.13089 ,  56.073298,
        57.015707,  57.958115,  58.900524,  59.842932,  60.78534 ,  61.727749,
        62.670157,  63.612565,  64.554974,  65.497382,  66.439791,  67.382199,
        68.324607,  69.267016,  70.209424,  71.151832,  72.094241,  73.036649,
        73.979058,  74.921466,  75.863874,  76.806283,  77.748691,  78.691099,
        79.633508,  80.575916,  81.518325,  82.460733,  83.403141,  84.34555 ,
        85.287958,  86.230366,  87.172775,  88.115183,  89.057592,  90.      ])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
Attributes:
    long_name:  latitude
    units:      degrees_north

In [36]:
# get just SH slice
ds_subset_1 = ds_1.isel(lat=slice(0,32))
ds_subset_2 = ds_2.isel(lat=slice(0,32))
ds_subset_3 = ds_3.isel(lat=slice(0,32))

## Section 4: Drop the pressure levels that we don't need

In [37]:
# look at pressure levels
ds_subset_1.lev

<xarray.DataArray 'lev' (lev: 32)>
array([  3.643466,   7.59482 ,  14.356632,  24.61222 ,  35.92325 ,  43.19375 ,
        51.677499,  61.520498,  73.750958,  87.82123 , 103.317127, 121.547241,
       142.994039, 168.22508 , 197.908087, 232.828619, 273.910817, 322.241902,
       379.100904, 445.992574, 524.687175, 609.778695, 691.38943 , 763.404481,
       820.858369, 859.534767, 887.020249, 912.644547, 936.198398, 957.48548 ,
       976.325407, 992.556095])
Coordinates:
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
Attributes:
    long_name:      hybrid level at midpoints (1000*(A+B))
    units:          hPa
    positive:       down
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    formula_terms:  a: hyam b: hybm p0: P0 ps: PS

In [38]:
ds_subset_1 = ds_subset_1.isel(lev=slice(20,31))

## Section 5: Keep only variables we need

In [39]:
T_subset = ds_subset_1[var_in_1]
TS_subset = ds_subset_2[var_in_2]
PS_subset = ds_subset_3[var_in_3]

In [40]:
T_subset.persist()
TS_subset.persist()
PS_subset.persist()

,Array,Chunk
Bytes,266.31 MiB,432.00 kiB
Shape,"(25, 303, 32, 288)","(1, 12, 32, 288)"
Count,775 Tasks,775 Chunks
Type,float32,numpy.ndarray


## Section 6: Interpolate the Temperature to set pressure levels

In [41]:
# We will need surface pressure and reference pressure for interpolation
# set a surface reference pressure [Pa]
p0 = 100000 

In [42]:
# grab parameters for interpolation
hyam = ds_subset_1["hyam"]
hybm = ds_subset_1["hybm"]

In [43]:
# Specify output pressure levels
new_levels = np.array([925, 850, 700])  # in millibars
new_levels = new_levels * 100  # convert to Pascals

In [44]:
%%time
T_subset_interp = gcomp.interp_hybrid_to_pressure(T_subset,
                                                  PS_subset,
                                                  hyam, hybm, p0 = p0,
                                                  new_levels=new_levels,
                                                  method='linear')

CPU times: user 32 ms, sys: 1.01 ms, total: 33 ms
Wall time: 35.5 ms


In [45]:
T_subset_interp.persist()

,Array,Chunk
Bytes,798.93 MiB,1.27 MiB
Shape,"(25, 303, 3, 32, 288)","(1, 12, 3, 32, 288)"
Count,775 Tasks,775 Chunks
Type,float32,numpy.ndarray


## Section 7: Calculate temperature gradient

- TS with 900,850,700 hPa

In [46]:
Tgrad = T_subset_interp - TS_subset

## Section 8: Save data for making composites

In [50]:
print(T_subset_interp.shape)
print(TS_subset.shape)
print(Tgrad.shape)

(25, 303, 3, 32, 288)
(25, 303, 32, 288)
(25, 303, 3, 32, 288)


In [51]:
# Flatten the times and member_id
T_subset_for_composites = T_subset_interp.stack(new=("member_id","time"))
TS_subset_for_composites = TS_subset.stack(new=("member_id","time"))
Tgrad_subset_for_composites = Tgrad.stack(new=("member_id","time"))

In [52]:
print(T_subset_for_composites.shape)
print(TS_subset_for_composites.shape)
print(Tgrad_subset_for_composites.shape)

(3, 32, 288, 7575)
(32, 288, 7575)
(3, 32, 288, 7575)


In [53]:
T_subset_for_composites

,Array,Chunk
Bytes,798.93 MiB,31.96 MiB
Shape,"(3, 32, 288, 7575)","(3, 32, 288, 303)"
Count,34342 Tasks,25 Chunks
Type,float32,numpy.ndarray


In [54]:
# assign to numpy array object
subsetarray_T = T_subset_for_composites.values
subsetarray_TS = TS_subset_for_composites.values
subsetarray_Tgrad = Tgrad_subset_for_composites.values

In [55]:
subsetarray_T.shape

(3, 32, 288, 7575)

## Section 9: Save data as a netcdf

### Tgrad

In [58]:
fout = 'antarctic_data_for_som_composites_TGRAD'

In [59]:
# set some info for output
units = 'K'
longname = 'temperature gradient (Taloft - Tsfc)'

In [61]:
subsetarray_Tgrad.shape

(3, 32, 288, 7575)

In [65]:
ds_to_save = xr.Dataset({'data': (['plev','lat','lon','training_times'], subsetarray_Tgrad)}, 
                        coords={'time':(['training_times'],Tgrad_subset_for_composites.time.values),
                                'member_id':(['training_times'],Tgrad_subset_for_composites.member_id.values),
                                'plev':(['plev'],Tgrad_subset_for_composites.plev.values),                                
                                'lon':(['lon'],Tgrad_subset_for_composites.lon.values),
                                'lat':(['lat'],Tgrad_subset_for_composites.lat.values)},
                        attrs={'Author': 'Alice DuVivier', 'units':units, 'longname':longname})

In [66]:
ds_to_save

<xarray.Dataset>
Dimensions:    (plev: 3, lat: 32, lon: 288, training_times: 7575)
Coordinates:
    time       (training_times) object 1980-07-16 12:00:00 ... 2080-09-16 00:...
    member_id  (training_times) object 'r1i1001p1f1' ... 'r9i1301p1f1'
  * plev       (plev) int64 92500 85000 70000
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... -62.67 -61.73 -60.79
Dimensions without coordinates: training_times
Data variables:
    data       (plev, lat, lon, training_times) float64 nan nan ... -15.61
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature gradient (Taloft - Tsfc)

In [67]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### T on plevels

In [68]:
fout = 'antarctic_data_for_som_composites_T'

In [69]:
# set some info for output
units = 'K'
longname = 'temperature aloft'

In [70]:
subsetarray_T.shape

(3, 32, 288, 7575)

In [71]:
ds_to_save = xr.Dataset({'data': (['plev','lat','lon','training_times'], subsetarray_T)}, 
                        coords={'time':(['training_times'],T_subset_for_composites.time.values),
                                'member_id':(['training_times'],T_subset_for_composites.member_id.values),
                                'plev':(['plev'],T_subset_for_composites.plev.values),                                
                                'lon':(['lon'],T_subset_for_composites.lon.values),
                                'lat':(['lat'],T_subset_for_composites.lat.values)},
                        attrs={'Author': 'Alice DuVivier', 'units':units, 'longname':longname})

In [72]:
ds_to_save

<xarray.Dataset>
Dimensions:    (plev: 3, lat: 32, lon: 288, training_times: 7575)
Coordinates:
    time       (training_times) object 1980-07-16 12:00:00 ... 2080-09-16 00:...
    member_id  (training_times) object 'r1i1001p1f1' ... 'r9i1301p1f1'
  * plev       (plev) int64 92500 85000 70000
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... -62.67 -61.73 -60.79
Dimensions without coordinates: training_times
Data variables:
    data       (plev, lat, lon, training_times) float64 nan nan ... 256.9 257.7
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature aloft

In [73]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### T Surface

In [74]:
fout = 'antarctic_data_for_som_composites_TS'

In [75]:
# set some info for output
units = 'K'
longname = 'surface temperature'

In [76]:
subsetarray_TS.shape

(32, 288, 7575)

In [77]:
ds_to_save = xr.Dataset({'data': (['lat','lon','training_times'], subsetarray_TS)}, 
                        coords={'time':(['training_times'],TS_subset_for_composites.time.values),
                                'member_id':(['training_times'],TS_subset_for_composites.member_id.values), 
                                'lon':(['lon'],TS_subset_for_composites.lon.values),
                                'lat':(['lat'],TS_subset_for_composites.lat.values)},
                        attrs={'Author': 'Alice DuVivier', 'units':units, 'longname':longname})

In [78]:
ds_to_save

<xarray.Dataset>
Dimensions:    (lat: 32, lon: 288, training_times: 7575)
Coordinates:
    time       (training_times) object 1980-07-16 12:00:00 ... 2080-09-16 00:...
    member_id  (training_times) object 'r1i1001p1f1' ... 'r9i1301p1f1'
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... -62.67 -61.73 -60.79
Dimensions without coordinates: training_times
Data variables:
    data       (lat, lon, training_times) float32 219.3 210.3 ... 273.5 273.3
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  surface temperature

In [79]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file